In [50]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('train.csv',index_col=0)

In [81]:
import sklearn.linear_model as lm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from neupy import algorithms, environment
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [73]:
y = dataset.species

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
if y.dtype == object:
    y = le.fit_transform(y)
else:
    pass

In [74]:
X = dataset.drop('species', axis=1)
X = (dataset.ix[:,1:].values).astype('float32')

In [87]:
#Splitting data to 5 sets and performing multinomial regression on each
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

log_loss_lr_total = []
log_loss_pnn_total = []
accuracy_score = []
i=0
environment.reproducible()
lgr = lm.LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg')
pnn = algorithms.PNN(std=10, verbose=False)
sgd = lm.SGDClassifier(loss='log')
rfc = RandomForestClassifier(max_depth=3, random_state=0)
knn = KNeighborsClassifier(n_neighbors=5)

for train_index, test_index in skf.split(X, y):    
    X_train = []
    X_test = []
    X_train, X_test, y_train, y_test = (X[train_index], X[test_index], y[train_index], y[test_index])
    
    # Multinomial linear regression
    lgr.fit(X_train, y_train)
    y_predicted = lgr.predict_proba(X_test)
    logLoss = log_loss(y_test, y_predicted)
    log_loss_lr_total.append(logLoss)
    
    # Stochastic Gradient Descent
    sgd.fit(X_train, y_train)
    y_predicted_sgd = sgd.predict_proba(X_test)
    logLossSGD = log_loss(y_test, y_predicted_sgd)
    
    #PNN classifier
    pnn.train(X_train, y_train)
    y_predicted_pnn = pnn.predict_proba(X_test)
    logLossPNN = log_loss(y_test, y_predicted_pnn)
    log_loss_pnn_total.append(logLossPNN)
    
    #Random Forest Classifier
    rfc.fit(X_train,y_train)
    y_predicted_rfc = rfc.predict_proba(X_test)
    logLossRFC = log_loss(y_test, y_predicted_rfc)
    
    #KNN classifier
    knn.fit(X_train, y_train)
    y_predicted_knn = knn.predict_proba(X_test)
    logLossKNN = log_loss(y_test, y_predicted_knn)
    
    #print(y_test)
    #print(y_predicted[0])
    #print(y_predicted_sgd[0])
    #print(y_predicted_pnn[0])
    #print(y_predicted_rfc[0])
    #print(y_predicted_knn[0])

    i += 1
    print("Log loss - Logistic regression for fold " + str(i) + "= " + str(logLoss) + "/n SGD  = " + str(logLossSGD) + "/n PNN = " +str(logLossPNN)
         + "/n RFC = " + str(logLossRFC) + "/n KNN = " + str(logLossKNN))
    
    
    #accuracy_score.append(metrics.accuracy_score(y_test, y_predicted_pnn))
    
log_loss_mean = np.mean(log_loss_lr_total)

#print("Average log loss - Logistic Regression is " + str(log_loss_mean))
log_loss_pnn_mean = np.mean(log_loss_pnn_total)
#print("Average log loss - PNN classifier is " + str(log_loss_pnn_mean))


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[ 3 49 65 94 84 40 54 78 53 89 98 16 74 50 58 31 43  4 75 44 83 84 13 66
 15  6 73 22 73 31 36 27 94 88 12 28 21 25 20 60 65 69 58 23 76 18 52 54
  9 48 47 64 81 83 36 21 81 20 62 88 34 92 79 82 32  4 35 72 60 71 72 52
 50 11 51 18 47  5  8 37 97 33  1 59  1 56  9 57 79 29 16 32 93 10 46 59
 76 15 10  0 69 51 39 62  2 24 26 35 25 87  0 55 34 38 45  7 93 56 38 75
 74 33 37 40 71 67 30 66 43 61 23 87 46 57  2 28 12 96 44 29 41 67 61 30
  5  3  6 85 39 14 98 17 42 63 86 86 80 78 14  8 26 13 77 80 92 45 41 17
 27 19 49 95 82 63 42 22 53 24 77 70 55 90 96 48 97 19 89 95 64 11 68 90
 91 70 85  7 91 68]
[0.01037408 0.00985452 0.00943888 0.01158202 0.0085934  0.00926718
 0.00958903 0.00993012 0.00981498 0.00986098 0.01072222 0.00979255
 0.01076085 0.00990468 0.00986265 0.00974823 0.00981153 0.00940073
 0.01058563 0.01057633 0.01021169 0.01020361 0.00994206 0.00965784
 0.01040188 0.00935955 0.01030709 0.0095074  0.01027142 0.00966383
 0.01020378 0.00977169 0.01051225 0.00876883 0.01007768 0.008

/opt/conda/lib/python3.6/site-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[84 58 20 84 36 54 20  1 54 15  4 51 94 36  1 21 51 81  9 60 31 83 50 65
 16 76 75 87 12 62 83 81  4 37 57 71 61  6 76 31 40 16 37 69 71 35 25  5
 39  9 44 60 14 13 87 18 78 78 30 14 35 46 21  8  6 92 38 40 15 32 93 92
 38 74 67 45 80 74 44 47 57 94 17 32 93 24 80 59 46 12 79 69  2 63 53 67
  0  2 42 10 52 59 22 86 43 75 24 86 45 98 66 73 73 79 56 41 25 27 18 63
  0  8 27 61 29 50 33 29 43 23 17 66 65 10 26 58 42 53 30 97 22 56 96 28
 82 90 64 28 34 88 89 26 88 41 98 70  7 52 70 13 91 95 23 49 55 19 85 64
 96 34 95 72 91 11 39  5 55 82 89 85 48 47  7 77 11 68 62  3 33 68 19 49
  3 90 72 77 48 97]
[0.0117618  0.01118657 0.01135858 0.00930418 0.01337278 0.01196179
 0.01147491 0.00822036 0.01135431 0.01121965 0.00990436 0.00891749
 0.00894144 0.00856057 0.00873885 0.01126217 0.01084346 0.00835944
 0.00908811 0.01005337 0.01080273 0.00948528 0.00937902 0.01150265
 0.00901156 0.01184977 0.00922018 0.00869058 0.00976047 0.0110604
 0.00974083 0.01174662 0.01120597 0.01290305 0.01138552 0.0124

/opt/conda/lib/python3.6/site-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


...

/opt/conda/lib/python3.6/site-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[84 51 15 51 36 20 21 37 31  8 16 84 46  8 36 44 16 40  6 60 60 28 31 83
 22 66 32 37 61 63 35 25 24 59 28 54 74 74  9 40 46 21 76 83 57 88 96 44
 82 63 24 54 43 12  5 52  1 73 78 47 69 18 29 53  1 45  9 13  2 66 59 79
  6 43 26 12 71 93 39 42 15 38 55 93 88 55 17 94 57 92 81 26 89 49 89 30
 58  4  4 76 71 13 62 61 25 18 62 14 78 65 50 58 38 87 50  7 69 23 65 41
 92 20 91 29 67  0 35 98 91 53 39 85 96 17  7 11 56 90 79 45 64 41 19 11
 10 95 10 42 68 14 22 27 98 72  3 19 75 81 67 56 47 75 94 95 32 52 87  3
 73 64 49 68 77 82  2 30 23 33 34 33 27 34 86  0 80 70 77 48  5 97 80 85
 90 97 70 48 72 86]
[0.01139749 0.0109956  0.01030068 0.00965661 0.01033061 0.0112763
 0.01136312 0.0086026  0.010867   0.01085763 0.01000317 0.0094023
 0.00971531 0.00912822 0.00910404 0.01041482 0.01094991 0.00881575
 0.00909994 0.01051081 0.01124109 0.00945839 0.00940507 0.01098286
 0.00886063 0.0101686  0.00954987 0.00857235 0.00930816 0.0111118
 0.00990944 0.01150141 0.01208394 0.01091694 0.01095787 0.009634

/opt/conda/lib/python3.6/site-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
Time: 0:00:00 | 50%|######################                      | ETA:  0:00:00

[51 16 36  8 84 36 28 37 84 44 24 46 44 83  1 15 40 55 60 74 21 54 16 12
 37 60 93 83 61 76 18  9  9 31 24 96 39 74 62 28 96  7  7 93 41 58 12 71
 91 43 61 20 63 26 11 31 57 62 66 19 13 75 50 32 47 43 94 68 47 66 35 88
 68 87 54 79 67 65 18  4 26 30 52 29 67 95 39 25 58 35 27 17 38 91 13 23
 79 77 22 49 98 46 63 53 20 25 78 10 65 33 41 98 71 95 52  3 29 69 51 27
 22 34  6 21 89 17 97 72 80 10 57 64 92 38 15 73 87 73 48 42 82 33 56  3
 42  1 53 55 90 19  6 30 64 49  2  8 45 76 92  0 23 69 59 80 90 32  5 59
 85 89 94 45 48 86 81 14  4 77 56 82  2 85 70 88  0 75 14 86 81 97 70 72
 34 40  5 11 78 50]
[0.01014771 0.0102623  0.00974067 0.01110505 0.00868414 0.01013696
 0.01071783 0.00824164 0.00939504 0.01032492 0.00963232 0.0088003
 0.00958669 0.00874615 0.00887501 0.0108651  0.00944835 0.00856222
 0.0098413  0.00996689 0.01060341 0.0098314  0.00997447 0.01051776
 0.01020626 0.00974867 0.00928704 0.00928227 0.01069564 0.01199727
 0.01117814 0.01033172 0.01387689 0.01054602 0.01093582 0.0087

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [91]:
#Predicting the value for the test data

test_dataset = pd.read_csv('test.csv',index_col=0)
y_test_dataset = dataset.species

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
if y_test_dataset.dtype == object:
    y_test_dataset = le.fit_transform(y)
else:
    pass

X_test_dataset = dataset.drop('species', axis=1)
X_test_dataset = (dataset.ix[:,1:].values).astype('float32')

y_pred_test = lgr.predict_proba(X_test_dataset)
log_loss_test = log_loss(y_test_dataset,y_pred_test)
y_pred_sgd_test = sgd.predict_proba(X_test_dataset)
log_loss_sgd_test = log_loss(y_test_dataset, y_pred_sgd_test)
y_pred_test_knn = knn.predict_proba(X_test_dataset)
log_loss_knn_test = log_loss(y_test_dataset,y_pred_test_knn)
y_predicted_test_pnn = pnn.predict_proba(X_test_dataset)
log_loss_pnn_test = log_loss(y_test_dataset, y_predicted_test_pnn)
y_predicted_rfc_test = rfc.predict_proba(X_test_dataset)
log_loss_rfc_test = log_loss(y_test_dataset, y_predicted_rfc_test)
print("Log loss - Logistic regression for fold " + str(i) + "= " + str(log_loss_test) + "/n SGD  = " + str(log_loss_sgd_test) + "/n PNN = " +str(logLossPNN)
         + "/n RFC = " + str(log_loss_rfc_test) + "/n KNN = " + str(log_loss_knn_test))
    

Time: 0:00:00 | 75%|#################################           | ETA:  0:00:00

Log loss - Logistic regression for fold 5= 4.155838008601256/n SGD  = 2.58176208834954/n PNN = 4.593970864959238/n RFC = 3.409768578504182/n KNN = 0.47876888110993376
